# BLAST-like Identity Comparison in Python
This notebook implements a simple BLAST-like algorithm for DNA sequence alignment from /Users/frvsbi/Documents/RIT/BIOL-530-630/Dry-Lab/Lab03/blast/blastn.pl

In [1]:
from Bio import SeqIO

def read_fasta(file_path):
    """Reads a FASTA file and returns a sequence string."""
    record = next(SeqIO.parse(file_path, "fasta"))  # Assume a single sequence
    return str(record.seq)

def find_word_hits(query, subject, word_size):
    """Finds exact word matches of size `word_size` between query and subject sequences."""
    word_hits = []
    query_len = len(query)
    
    # Create a dictionary of words from query
    query_words = {query[i:i+word_size]: i for i in range(query_len - word_size + 1)}
    
    # Scan subject for matching words
    for j in range(len(subject) - word_size + 1):
        word = subject[j:j+word_size]
        if word in query_words:
            word_hits.append((query_words[word], j))  # Store (query_pos, subject_pos)
    
    return word_hits

def extend_alignment(query, subject, q_pos, s_pos, match_score=5, mismatch_score=-2):
    """Extends alignment from a word hit and returns the best local alignment score."""
    score = 0
    aligned_q, aligned_s = "", ""
    
    # Extend forward
    i, j = q_pos, s_pos
    while i < len(query) and j < len(subject):
        if query[i] == subject[j]:
            score += match_score
        else:
            score += mismatch_score
        aligned_q += query[i]
        aligned_s += subject[j]
        i += 1
        j += 1
    
    return score, aligned_q, aligned_s

def blast_simulation(query_file, subject_file, word_size=10):
    """Runs a BLAST-like identity comparison between two DNA sequences."""
    query_seq = read_fasta(query_file)
    subject_seq = read_fasta(subject_file)
    
    word_hits = find_word_hits(query_seq, subject_seq, word_size)
    
    best_alignment = None
    best_score = float('-inf')
    
    for q_pos, s_pos in word_hits:
        score, aligned_q, aligned_s = extend_alignment(query_seq, subject_seq, q_pos, s_pos)
        if score > best_score:
            best_score = score
            best_alignment = (aligned_q, aligned_s, score)
    
    if best_alignment:
        print("Best Alignment:")
        print(best_alignment[0])
        print(best_alignment[1])
        print("Score:", best_alignment[2])
    else:
        print("No significant alignments found.")

In [3]:
# Example usage
# blast_simulation("seq1.fasta", "seq2.fasta")
blast_simulation("/Users/frvsbi/Documents/RIT/BIOL-530-630/Dry-Lab/Lab03/blast/blastdb/query1.fasta", "/Users/frvsbi/Documents/RIT/BIOL-530-630/Dry-Lab/Lab03/blast/blastdb/database.fasta")

Best Alignment:
TGGTACTATTACCCGAGGTTAACGATCCTAGGTTTAGTAAGTTGGTCGAAATTGATTGTTTATAGAGAAAATATCTATAATAAATTTAATCAGTAATTTAAAAACAGGTCCGTATTTTAGCTTTCAAACCTTAAAAACTCCCAAAGTATGTGCTCTACCAAGCTAAGTTATATCCTGAATATAAAAGTAACAAAGAACGAACTTTCCTCTAAACGATTTAATCTAGAAAACTTGACAAATGACAATAAAAAAGATGATGATAAGAGTATTTTAACAGATCTCTCTCAAACAAATTTATGGTCTATTTCTTTCGCTTTTTATTTCTTTTAATCAATATTGATCAAGAAACTGTTTTAAATTGGTTTGGAATTGATAATCCAAATAACGTAAATGTTTTATCCGAGAATGTTACCATTTCAAGTAGCCAAACTACGACAGCTCCTGCTGTTTCGTTTCAAACATTGAGTTTAATTATAGAAGGTCTCTGGCAACATGTTCAAGATGGATTAACTTTAGCAGATATTGAAAATCTGTTATTTTTTATTCTCTTTATTCGATTTGTGATTTTAGCAATTCGTTATAATTTAAAAACCTCATTTTATATTACTTGTATTGGACTTTTTGCAGGATATCTTTGGTATCGACATCTCATTGATTTAATTGCTATGTACCGAAGTATGTTGATTAAACTTCCATATTTACATAAATTAGGAATTGATGCTATTCAACTAAGTTCTATGAGTCGTCAAATTGTGCTAACCGACTTAAAACTTGGTGAGAATGTTCACTGGTACAATCCTGGTCAAGTCTTATATTATGCCATTTTAAAAGGAATTATTCAAGTTGATCCCGAAACGGGACTTCGCTATTATATTGATCCACTTTCAATGCTCATTTCTAATTTGAAAGAATCTCAAAAAGAAACTATTTTACCAATTTACTATAAAGTATACAATAAGATTATTCCTCAAGTTTTTGAAGGAA